# Data Crawling
For the training of ML1 and ML2 we need data. In this project the selection of training data is an important creative decision. Since Machine Learning algorithms try to minimize some kind of error, it is crutial how this error is defined. The networks will try to produce sound as close as possible to their training data. Thus the produced sounds will be very different, for different training data. As stated above, we want to give as many creative decision as possible to machines, since the goal is a meta-human sound. Therefor we've decided to use yet another machine learning algorithm to select the training data: YouTube's recomendation algorithm.

The following code controls the mouse of the users system to collect YouTube links. For this the user has to open a YouTube video on the platform on a Full-HD screen. The code will automatically click on the next recomendation and copy the URL. That can be used to download the sound of the YouTube video, e.g. via the program MediaHuman (https://www.mediahuman.com/de/).

WARNING: This code controls the mouse! You can only stop it by pressing and holding down the space bar!

In [ ]:
import mouse, keyboard
import time

def click_next_video():
    mouse.move(504, 700)
    time.sleep(5)
    mouse.click()
    time.sleep(2)
    keyboard.send("shift+n")
    time.sleep(1)

def copy_link():
    mouse.move(504, 50)
    mouse.click()
    time.sleep(0.6)
    keyboard.send("ctrl+c")

if __name__ == '__main__':

    # while True:
    #    print(mouse.get_position())
    time.sleep(5) # time to change the window to Youtube

    for i in range(1000):
        if keyboard.is_pressed("space"):
            exit()
        copy_link()
        click_next_video()
        print(i)



During testing of this project, we've recognized that songs with full instrumentation are too complex. Rather we want to use single instruments or vocals. Again, we don't need a human to decide which signal counts as "vocals". We can use another machine learning algorithm to extract stems from the downloaded song. We used spleeter for this (more info here: https://github.com/deezer/spleeter)


In [ ]:
import time
import librosa
import soundfile as sf
from spleeter.separator import Separator
import glob
import numpy as np
import os


if __name__ == '__main__':
   
    num = 50
    step_size = 450

    separator = Separator('spleeter:4stems')
    path = os.path.join('demo_data/', "*")
    list_of_files = glob.glob(path)
    start_time = time.time()

    list_of_files = list_of_files[num:num+step_size]

    for i,file in enumerate(list_of_files):
        separator.separate_to_file(file,
                                   "demo_data/stems",
                                   filename_format="{instrument}/{instrument}" + str(i+num) + ".{codec}",
                                   synchronous=False)
    separator.join()


Spleeter splits each song in 4 stems: vocals, bass, drums and other. Intrestingly the results are not always perfect. For example on a song that has flutes, but no vocals, the neural network seperates the flutes and writes the data on the vocal track. But the network is not always trying to find a vocals stem. It seems to be able, to recognize if there are no vocals in some cases. On songs without a strong leading voice, the vocals file can be empty.

Here we as the programmers have to make a decision which data to keep. As our goal is meta-human music, we want to keep all vocals stems that are not empty. We can keep the machine interpretation of what vocals are, and do not have to correct that with our human interpretation. 
We delete the empty files (or nearly empty files) in the next step.